In [1]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
from PIL import Image
import os

input_size = (224, 224)

# Path to your dataset folders (normal, dislocation, fractured)
dataset_path = 'D:/DT MP'

# Path to save resized and converted RGB images
output_path = 'D:/DTMP'

for folder in os.listdir(dataset_path):
    if os.path.isdir(os.path.join(dataset_path, folder)):
        for filename in os.listdir(os.path.join(dataset_path, folder)):
            if filename.lower().endswith(('.kytjpg', '.jpg')):
                image_path = os.path.join(dataset_path, folder, filename)
                img = Image.open(image_path).convert("RGB")  # Convert to RGB format
                img = img.resize(input_size, Image.ANTIALIAS)
                img.save(os.path.join(output_path, folder, filename))


C:\Users\swapn\AppData\Local\Temp\ipykernel_1524\126666604.py:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(input_size, Image.ANTIALIAS)


In [6]:
base_dir = 'D:/FDT'
os.makedirs(os.path.join(base_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'test'), exist_ok=True)

In [7]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15


In [8]:
import random
import shutil

for class_name in os.listdir('D:/DTMP'):
    class_path = os.path.join('D:/DTMP', class_name)
    images = os.listdir(class_path)
    num_images = len(images)
    random.shuffle(images)

    train_split = int(train_ratio * num_images)
    val_split = int(val_ratio * num_images)

    # Copy images to the corresponding split directories
    for i, image in enumerate(images):
        src_path = os.path.join(class_path, image)
        if i < train_split:
            dst_dir = os.path.join(base_dir, 'train', class_name)
        elif i < train_split + val_split:
            dst_dir = os.path.join(base_dir, 'val', class_name)
        else:
            dst_dir = os.path.join(base_dir, 'test', class_name)
        
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy(src_path, os.path.join(dst_dir, image))


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [10]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    'D:/FDT/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    'D:/FDT/val',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'D:/FDT/test',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 747 images belonging to 3 classes.
Found 160 images belonging to 3 classes.
Found 162 images belonging to 3 classes.


In [11]:
model = Sequential()

# Convolutional Layers
model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(256, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(384, (3, 3), activation='relu'))
model.add(Conv2D(384, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))  # 3 classes: fractured, normal, dislocation


In [13]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

num_epochs = 30
history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator
)


Epoch 1/30
24/24 [==============================] - 54s 2s/step - loss: 0.8527 - accuracy: 0.6613 - val_loss: 0.8381 - val_accuracy: 0.6625
Epoch 2/30
24/24 [==============================] - 54s 2s/step - loss: 0.8038 - accuracy: 0.6613 - val_loss: 0.8080 - val_accuracy: 0.6625
Epoch 3/30
24/24 [==============================] - 43s 2s/step - loss: 0.7535 - accuracy: 0.6613 - val_loss: 0.7732 - val_accuracy: 0.6625
Epoch 4/30
24/24 [==============================] - 41s 2s/step - loss: 0.6977 - accuracy: 0.6747 - val_loss: 0.6270 - val_accuracy: 0.7063
Epoch 5/30
24/24 [==============================] - 41s 2s/step - loss: 0.6375 - accuracy: 0.7028 - val_loss: 0.6470 - val_accuracy: 0.7312
Epoch 6/30
24/24 [==============================] - 40s 2s/step - loss: 0.6582 - accuracy: 0.7229 - val_loss: 0.6531 - val_accuracy: 0.7375
Epoch 7/30
24/24 [==============================] - 40s 2s/step - loss: 0.5056 - accuracy: 0.7510 - val_loss: 0.4307 - val_accuracy: 0.7812
Epoch 8/30
24/24 [==

In [14]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


6/6 [==============================] - 1s 224ms/step - loss: 0.1684 - accuracy: 0.9383
Test accuracy: 93.83%
